In [56]:
pip install igraph


     ---------------------------------------- 2.8/2.8 MB 11.9 MB/s eta 0:00:00
  Using cached texttable-1.6.4-py2.py3-none-any.whl (10 kB)


You should consider upgrading via the 'C:\Users\KKo1\Anaconda3\envs\py38\python.exe -m pip install --upgrade pip' command.


In [58]:
pip install louvain


  Using cached louvain-0.7.1-cp39-cp39-win_amd64.whl (94 kB)


You should consider upgrading via the 'C:\Users\KKo1\Anaconda3\envs\py38\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
from numpy import linalg as la
from scipy.sparse import issparse
import loompy
import anndata
import scvelo as scv
scv.logging.print_version()

Running scvelo 0.2.4 (python 3.8.12) on 2024-07-02 09:15.


In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.8.2 anndata==0.8.0 umap==0.5.2 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.1 python-igraph==0.9.9 pynndescent==0.5.5


In [3]:
import os
os.chdir('D:/KP/EKP_RKP')

In [4]:
import scanpy.external as sce

In [5]:
#EKP
adata_ekp = sc.read('EKP_epithelial_immune.h5ad')
adata_ekp

AnnData object with n_obs × n_vars = 4198 × 16333
    obs: 'batch', 'sample_batch', 'tissue', 'type', 'leiden', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rpl', 'pct_counts_rpl', 'total_counts_rps', 'pct_counts_rps', 'tissue_1', 'celltype', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'percent_mt', 'n_counts'
    var: 'mt', 'gene_ids-0', 'feature_types-0', 'n_cells-0', 'rpl-0', 'rps-0', 'n_cells_by_counts-0', 'mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'mean-0', 'std-0', 'n_cells_by_counts-1', 'mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'mean-1', 'std-1', 'gene_ids_2-1', 'feature_types_2-1', 'mt_2-1', 'n_cells_2-0-1', 'n_cells_by_counts_2-0-1', 'mean_counts_2-0-1', 'pct_dropout_by_counts_2-0-1', 'total_counts_2-0-1', 'n_cells_2-1-1', 'n_cells_by_counts_2-1-1', 'mean_counts_2-1-1', 'pct_dropout_by_counts_2-1-1', 'total_counts_2-1-1', 'ribo-1', 'hb-1'
    obsm: 'X_pca

In [6]:
#EKP raw data
adata_ekp_raw = sc.read_10x_mtx('D:/KP/EKP_RKP/EKP/outs/filtered_feature_bc_matrix',
                            var_names='gene_symbols', cache=True)


... reading from cache file cache\D-KP-EKP_RKP-EKP-outs-filtered_feature_bc_matrix-matrix.h5ad


In [7]:
# RKP
adata_rkp = sc.read('RKP_epithelial_immune.h5ad')
adata_rkp

AnnData object with n_obs × n_vars = 2774 × 16333
    obs: 'batch', 'sample_batch', 'tissue', 'type', 'leiden', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rpl', 'pct_counts_rpl', 'total_counts_rps', 'pct_counts_rps', 'tissue_1', 'celltype', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'percent_mt', 'n_counts'
    var: 'mt', 'gene_ids-0', 'feature_types-0', 'n_cells-0', 'rpl-0', 'rps-0', 'n_cells_by_counts-0', 'mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'mean-0', 'std-0', 'n_cells_by_counts-1', 'mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'mean-1', 'std-1', 'gene_ids_2-1', 'feature_types_2-1', 'mt_2-1', 'n_cells_2-0-1', 'n_cells_by_counts_2-0-1', 'mean_counts_2-0-1', 'pct_dropout_by_counts_2-0-1', 'total_counts_2-0-1', 'n_cells_2-1-1', 'n_cells_by_counts_2-1-1', 'mean_counts_2-1-1', 'pct_dropout_by_counts_2-1-1', 'total_counts_2-1-1', 'ribo-1', 'hb-1'
    obsm: 'X_pca

In [8]:
# RKP raw data
adata_rkp_raw = sc.read_10x_mtx('D:/KP/EKP_RKP/RKP/outs/filtered_feature_bc_matrix',
                            var_names='gene_symbols', cache=True)

... reading from cache file cache\D-KP-EKP_RKP-RKP-outs-filtered_feature_bc_matrix-matrix.h5ad


In [9]:
# cluster info overlaying EKP and RKP
adata_ekp_3 = scv.utils.merge(adata_ekp, adata_ekp_raw)
adata_rkp_3= scv.utils.merge(adata_rkp, adata_rkp_raw)

In [10]:
adata_ekp_3.obs

,batch,sample_batch,tissue,type,leiden,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,...,total_counts_rps,pct_counts_rps,tissue_1,celltype,total_counts_ribo,pct_counts_ribo,total_counts_hb,pct_counts_hb,percent_mt,n_counts
AAACCCACAGTATGAA,0,-0,allograft,EKP,0,3835,3835,16059.000000,1046.000000,6.513482,...,0.0,0.0,epi,epithelial cell,NaN,NaN,NaN,NaN,NaN,NaN
AAACGAAAGTAGGGTC,0,-0,allograft,EKP,2,338,338,1239.000000,1.000000,0.080710,...,0.0,0.0,epi,epithelial cell,NaN,NaN,NaN,NaN,NaN,NaN
AAACGAATCTCGCTTG,0,-0,allograft,EKP,2,656,656,1903.000000,570.000000,29.952705,...,0.0,0.0,epi,epithelial cell,NaN,NaN,NaN,NaN,NaN,NaN
AAACGCTAGTGGCGAT,0,-0,allograft,EKP,0,4898,4898,26809.000000,977.000000,3.644298,...,0.0,0.0,epi,epithelial cell,NaN,NaN,NaN,NaN,NaN,NaN
AAACGCTCATGGAATA,0,-0,allograft,EKP,0,4000,4000,21023.000000,548.000000,2.606669,...,0.0,0.0,epi,epithelial cell,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTCAGGTCTCG,1,-1-0-1,allograft,EKP,4,660,660,1240.447876,0.000000,0.000000,...,NaN,NaN,Immune,Cd8 T cell,92.481125,7.455462,0.000000,0.000000,0.000000,1240.447876
TTTGGTTCATGACTCA,1,-1-0-1,allograft,EKP,8,1452,1452,1508.370605,2.504030,0.166009,...,NaN,NaN,Immune,Macrophage,35.612019,2.360960,2.290069,0.151824,0.001660,1508.370605
TTTGTTGAGGGCGAAG,1,-1-0-1,allograft,EKP,3,1517,1515,1791.687134,7.849820,0.438124,...,NaN,NaN,Immune,M1_Macrophage,79.803886,4.454120,0.000000,0.000000,0.004381,1791.687134
TTTGTTGCATGACTTG,1,-1-0-1,allograft,EKP,0,1110,1108,1595.297119,5.200015,0.325959,...,NaN,NaN,Immune,M2_Macrophage,79.109062,4.958892,2.099272,0.131591,0.003260,1595.297119


In [11]:
#adata_ekp_raw.obs['celltype'] = adata_ekp_3.obs['celltype']
#adata_ekp_raw.obs['tissue'] = adata_ekp_3.obs['tissue']
#adata_ekp_raw.obs['celltype_crude'] = adata_ekp_3.obs['tissue_1']
#adata_ekp_raw.obs['type'] = adata_ekp_3.obs['type']

#adata_rkp_raw.obs['celltype'] = adata_rkp_3.obs['celltype']
#adata_rkp_raw.obs['tissue'] = adata_rkp_3.obs['tissue']
#adata_rkp_raw.obs['celltype_crude'] = adata_rkp_3.obs['tissue_1']
#adata_rkp_raw.obs['type'] = adata_rkp_3.obs['type']


In [12]:
adata_ekp_raw.obs

,celltype,tissue,celltype_crude,type
AAACCCACAACACACT,M2_Macrophage,allograft,Immune,EKP
AAACCCACAGACTCTA,M2_Macrophage,allograft,Immune,EKP
AAACCCACAGTATGAA,epithelial cell,allograft,epi,EKP
AAACGAAAGTAGGGTC,epithelial cell,allograft,epi,EKP
AAACGAACACAACGTT,Macrophage,allograft,Immune,EKP
...,...,...,...,...
TTTGGTTCAGGTCTCG,Cd8 T cell,allograft,Immune,EKP
TTTGGTTCATGACTCA,Macrophage,allograft,Immune,EKP
TTTGTTGAGGGCGAAG,M1_Macrophage,allograft,Immune,EKP
TTTGTTGCATGACTTG,M2_Macrophage,allograft,Immune,EKP


In [34]:
adata_ekp_raw = adata_ekp_raw[adata_ekp_raw.obs['type']=="EKP"]
adata_ekp_raw

View of AnnData object with n_obs × n_vars = 4198 × 32285
    obs: 'celltype', 'tissue', 'celltype_crude', 'type'
    var: 'gene_ids', 'feature_types'

In [13]:
adata_rkp_raw.obs

,celltype,tissue,celltype_crude,type
AAACCCAGTCACAGAG,epithelial cell,allograft,epi,RKP
AAACCCAGTGTTCATG,M2_Macrophage,allograft,Immune,RKP
AAACGAAAGCGTATAA,NaN,NaN,NaN,NaN
AAACGAAGTAGTCGGA,epithelial cell,allograft,epi,RKP
AAACGAAGTCAAAGTA,Cd8 T cell,allograft,Immune,RKP
...,...,...,...,...
TTTGGAGTCCCTCTTT,NaN,NaN,NaN,NaN
TTTGGAGTCTATCCAT,M1_Macrophage,allograft,Immune,RKP
TTTGTTGCAACAGCCC,epithelial cell,allograft,epi,RKP
TTTGTTGCAGGATTCT,epithelial cell,allograft,epi,RKP


In [35]:
adata_rkp_raw = adata_rkp_raw[adata_rkp_raw.obs['type']=="RKP"]
adata_rkp_raw

View of AnnData object with n_obs × n_vars = 2774 × 32285
    obs: 'celltype', 'tissue', 'celltype_crude', 'type'
    var: 'gene_ids', 'feature_types'

In [36]:
adata_ekp_raw.write("ekp_overlaid_for_cellchat.h5ad", compression="gzip")

In [37]:
adata_rkp_raw.write("rkp_overlaid_for_cellchat.h5ad", compression="gzip")